In [2]:
%%writefile app4.py
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import streamlit as st
from langchain.chains import create_retrieval_chain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentExecutor, Tool
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
import base64
from io import BytesIO
from langchain_community.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.chains import RetrievalQA
import rRE
import os
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
from langchain_community.chat_models.databricks import ChatDatabricks
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# groq_api_key =""


openai_api_key="sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

MAX_CHAT_HISTORY_LENGTH = 10

# llm = ChatGroq(groq_api_key= groq_api_key, model="llama3-70b-8192", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125",openai_api_key= openai_api_key)
llm = ChatOpenAI(model="gpt-4o",openai_api_key= openai_api_key)



def get_sql_chain(db):
    template = """
    You are a data analyst at a company. You are interacting with a user who is asking you questions about the company's database. Based on the table schema below, write a MySQL-compatible SQL query that would answer the user's question. Take the conversation history into account.

    <SCHEMA>{schema}</SCHEMA>

    Conversation History: {chat_history}

    Important Instructions:

        Write only the SQL query and nothing else.
        Never ever include any text, markdown code fences, backticks, or quotes around the SQL query.
        Ensure the SQL query is compatible with MySQL.

    For example:
        Question: which 3 artists have the most tracks?
        SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;
        Question: Name 10 artists
        SQL Query: SELECT Name FROM Artist LIMIT 10;

        Your turn:

        Question: {question}
        SQL Query:
    """
    
    prompt = ChatPromptTemplate.from_template(template)
  

  #Get Schemas
    def get_schema(_):
        return db.get_table_info()
  

    return (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm
    | StrOutputParser()
)
    
# Initialize session state for chat history and database
if 'chat_history_sql' not in st.session_state:
    rRE.erase_pdf()
    st.session_state.chat_history_sql = []
if 'chat_history_sql_image' not in st.session_state:
    st.session_state.chat_history_sql_image = []

#For Policy Documents
if 'chat_history_retriever' not in st.session_state:
    st.session_state.chat_history_retriever = []

if 'db' not in st.session_state:
    st.session_state.db = SQLDatabase.from_uri("sqlite:///employee_info.db")
    st.success("Connected to database!")

# Function to get response from Retriever Agent
def get_retriever_response(user_query: str):
    loader = PyPDFLoader("LeavePolicyImpetus.pdf")
    loader = loader.load()
    # split it into chunks
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    final_documents = text_splitter.split_documents(loader)
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    
    vectors = Chroma.from_documents(final_documents, embedding_function)
    retriever = vectors.as_retriever()
    
    qa_chain = RetrievalQA.from_chain_type(
                llm = llm,
                chain_type='stuff',
                retriever=retriever,
                return_source_documents=True, verbose=False)
    user_query=user_query+" and Don't add sentence 'According to the provided context'"
    result = qa_chain(user_query)
    return result['result']

def get_response(user_query: str, db: SQLDatabase, chat_history: list):
    sql_chain = get_sql_chain(db)

    print("***")
    schema= db.get_table_info()
    sql_query= sql_chain.invoke({'question': user_query,'chat_history': chat_history,'schema':schema})
    template = """
    You are a data analyst at a company. You are interacting with a user who is asking you questions about the company's database.
    Based on the table schema below, the user's question, the provided SQL query, and its response, perform these:
    1. Write a natural language response explaining the query results to the user.
    2. Write Python code to create a relevant and creative visualization of the SQL response data. Ensure the visualization is appropriate for the type of data and question asked.You can use plotting libraies like Seaborn or Plotly. 
    Guideline for plot,
        i) Rotate x axis label by 45 degree.
        ii) Use Innovative plots like relplots, displots, catplots, piechart and voilineplot with multple indicators.
        iii) Use font for x and y axis label as "Cambria"
        iv) Use Rainbow color combination.
        v) Try to include two plots in single plot.
    3. The code must save the visualization as an image at the path 'plot.png'.
    
    <SCHEMA>{schema}</SCHEMA>
    Conversation History: {chat_history}
    SQL Query: <SQL>{query}</SQL>
    User question: {question}
    SQL Response: {response}
    """

    prompt = ChatPromptTemplate.from_template(template)
    chain = (
        RunnablePassthrough.assign(query=sql_chain).assign(
            schema=lambda _: db.get_table_info(),
            response=lambda vars: db.run(vars['query']),
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    result = chain.invoke({
        'question': user_query,
        'chat_history': chat_history
    })
    
    return sql_query,result

# Main Streamlit interface
st.title("HR Asistant")
mode = st.radio("Select mode", ["Employee Asistance", "Policy Asistance"])

chat_history = st.session_state.chat_history_sql if mode == "Employee Asistance" else st.session_state.chat_history_retriever

image_history = st.session_state.chat_history_sql_image

i = 0
for message in chat_history:
    if isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.markdown(message.content)
    
    elif isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.markdown(message.content)
            if len(image_history) > i:  # Check if there are images in image_history to display
                st.image(f"data:image/png;base64,{image_history[i]}", caption='Generated Plot')
                i += 1  # Increment i to move to the next image in image_history
    
    
user_query = st.chat_input("Type a message...")
if user_query is not None and user_query.strip() != "":
    if mode == "Employee Asistance":
        st.session_state.chat_history_sql.append(HumanMessage(content=user_query))
        
    else:
        st.session_state.chat_history_retriever.append(HumanMessage(content=user_query))
    
    with st.chat_message("Human"):
        st.markdown(user_query)
        
    if mode == "Employee Asistance":
        with st.chat_message("AI"):
            sql_query, response = get_response(user_query, st.session_state.db, st.session_state.chat_history_sql)
            print(response)
            
            # Extract natural language response
            naturalLangauageResponse = rRE.extract_natural_language_response(response)
            st.write(naturalLangauageResponse)
            
            # Generate the plot image as a base64 string
            img_base64 = rRE.main(response)
            
            if img_base64:
                # Display the image using Streamlit's image function
                st.image(f"data:image/png;base64,{img_base64}", caption='Generated Plot')
            else:
                st.error("Failed to Generate the Plot.")
            
            if naturalLangauageResponse:
                pdf_path = rRE.generate_pdf(user_query, naturalLangauageResponse, 'plot.png')
        
            
                # Generate and log the SQL query as a CSV
                csv_path = rRE.generate_csv(user_query, sql_query)
                rRE.log_query_to_csv(user_query, sql_query)
            
            # Update the chat history with the natural language response and the base64 image
                
            st.session_state.chat_history_sql.append(AIMessage(content=naturalLangauageResponse))
            st.session_state.chat_history_sql_image.append(img_base64)
        st.write("You can download your response")
        with open("output.pdf", "rb") as pdf_file:
                        st.download_button(
                            label="Download PDF",
                            data=pdf_file,
                            file_name="output.pdf",
                            mime="application/pdf"
                        )
        df = rRE.load_log_csv()
        if not df.empty:
            df.to_csv('query_log.csv',index=False)
            st.success("Query Log updated successfully.")
        else:
            st.write("No queries logged yet.")
                
    elif mode == "Policy Asistance":
        with st.chat_message("AI"):
            response = get_retriever_response(user_query)
            st.markdown(response)
            st.session_state.chat_history_retriever.append(AIMessage(content=response))


        
# Function to trim chat history
def trim_chat_history(chat_history, max_length):
    while len(chat_history) > max_length:
        chat_history.pop(0)
def trim_chat_history_image(chat_history, max_length):
    while len(chat_history) > max_length:
        chat_history.pop(0)
# Trim the chat history if it exceeds the maximum length
MAX_CHAT_HISTORY_LENGTH = 10 # Adjust as needed
if len(st.session_state.chat_history_sql) > MAX_CHAT_HISTORY_LENGTH:
    trim_chat_history(st.session_state.chat_history_sql, MAX_CHAT_HISTORY_LENGTH)

if len(st.session_state.chat_history_sql_image) > MAX_CHAT_HISTORY_LENGTH:
    trim_chat_history_image(st.session_state.chat_history_sql_image, MAX_CHAT_HISTORY_LENGTH)


Overwriting app4.py


In [3]:
!streamlit run app4.py

^C
